### Merging datasets generated in different steps

In [20]:
ds_argilla = load_dataset("argilla/OpenHermes-2.5-dpo-row-index-v2", split="train")

Generating train split:   0%|          | 0/805335 [00:00<?, ? examples/s]

In [21]:
ds_to_update = load_dataset("vwxyzjn/OpenHermes-2.5-dpo-with-system-ckpt-2", split="train")

Generating train split:   0%|          | 0/192649 [00:00<?, ? examples/s]

In [22]:
cols = ['conversations', 'input', 'generation_model', 'generation_prompt', 'raw_generation_responses', 'generations']

ds_argilla_updated = Dataset.from_pandas(
    pd.concat(
        [
            ds_argilla.select_columns(cols).to_pandas(),
            ds_to_update.select_columns(cols).to_pandas()
        ],
        axis=0
    ),
    preserve_index=False
)

In [23]:
ds_argilla_updated

Dataset({
    features: ['conversations', 'input', 'generation_model', 'generation_prompt', 'raw_generation_responses', 'generations'],
    num_rows: 997984
})

In [8]:
from datasets import load_dataset
import multiprocessing

og_ds = load_dataset("teknium/OpenHermes-2.5", split="train")
og_ds = og_ds.add_column("row_index", list(range(len(og_ds))))
#long_conversations = og_ds.filter(lambda row: len(row["conversations"]) > 2, num_proc=multiprocessing.cpu_count())

def merge_with_system(row):
    conversations = row["conversations"]
    if conversations[0]["from"] == "system":
        conversations[1]["value"] = conversations[0]["value"] + " " + conversations[1]["value"]
        conversations = conversations[1:]

    return {"conversations": conversations, "input": conversations[0]["value"]}

ds_merged = og_ds.map(merge_with_system, num_proc=multiprocessing.cpu_count())
ds_merged = ds_merged.map(lambda x: {"conv-str": str(x["conversations"])}, num_proc=multiprocessing.cpu_count())

ds_filtered = ds_merged.filter(lambda row: len(row["conversations"]) < 3, num_proc=multiprocessing.cpu_count())

# TO REMOVE DUPLICATES AT THE CONVERSATION LEVEL, UNUSED FOR THE GENERAL DATASET
#df_filtered = ds_filtered.to_pandas().drop_duplicates(subset="input")
#from datasets import Dataset
#ds_filtered = Dataset.from_pandas(df_filtered, preserve_index=False)

#ds_filtered.select(range(192648)).push_to_hub("argilla/OpenHermes-2.5-with-system-1", split="train")
#ds_filtered.select(range(192648, len(filtered_res))).push_to_hub("argilla/OpenHermes-2.5-with-system-2", split="train")

In [27]:
ds_argilla_updated_with_str = ds_argilla_updated.map(lambda x: {"conv-str": str(x["conversations"])}, num_proc=1)

Map:   0%|          | 0/997984 [00:00<?, ? examples/s]

In [29]:
df_argilla = ds_argilla_updated_with_str.to_pandas()
#df_filtered = og_ds_with_str.to_pandas()
df_filtered = ds_filtered.to_pandas()

df_argilla["row_index"] = list(range(len(df_argilla)))
df_filtered["row_index"] = list(range(len(df_filtered)))

df_merged = df_filtered[["row_index", "conv-str"]].merge(df_argilla, on="conv-str")
del df_argilla, df_filtered
df_merged = df_merged.rename(columns={"row_index_x": "index_filtered", "row_index_y": "index_argilla"})


In [30]:
df_merged.shape

(997984, 9)

In [11]:
df_merged.head()

,index_filtered,conv-str,conversations,input,generation_model,generation_prompt,raw_generation_responses,generations,index_argilla
0,0,"[{'from': 'human', 'value': 'Every day, a tree...","[{'from': 'human', 'value': 'Every day, a tree...","Every day, a tree drops 7 leaves. How many lea...",[NousResearch/Nous-Hermes-2-Yi-34B],"[<|im_start|>system\nYou are a helpful, respec...","[First, let's convert February in a non-leap y...","[First, let's convert February in a non-leap y...",0
1,1,"[{'from': 'human', 'value': 'In analytical che...","[{'from': 'human', 'value': 'In analytical che...","In analytical chemistry, what is the principle...",[NousResearch/Nous-Hermes-2-Yi-34B],"[<|im_start|>system\nYou are a helpful, respec...",[A. It compensates for variations in sample pr...,[A. It compensates for variations in sample pr...,1
2,2,"[{'from': 'human', 'value': 'A rectangular gar...","[{'from': 'human', 'value': 'A rectangular gar...",A rectangular garden has a length of 25 feet a...,[NousResearch/Nous-Hermes-2-Yi-34B],"[<|im_start|>system\nYou are a helpful, respec...",[To determine the amount of fencing needed to ...,[To determine the amount of fencing needed to ...,2
3,3,"[{'from': 'human', 'value': 'What was the purp...","[{'from': 'human', 'value': 'What was the purp...",What was the purpose of the Colosseum in Rome?,[NousResearch/Nous-Hermes-2-Yi-34B],"[<|im_start|>system\nYou are a helpful, respec...",[The primary purpose of the Colosseum in Rome ...,[The primary purpose of the Colosseum in Rome ...,3
4,4,"[{'from': 'human', 'value': ""A man claims he c...","[{'from': 'human', 'value': 'A man claims he c...",A man claims he can predict the outcome of a d...,[NousResearch/Nous-Hermes-2-Yi-34B],"[<|im_start|>system\nYou are a helpful, respec...",[Since we already know that the man is able to...,[Since we already know that the man is able to...,4


In [31]:
ds_argilla_new = Dataset.from_pandas(df_merged, preserve_index=False)
ds_argilla_new = ds_argilla_new.rename_column("index_filtered", "row_index").select_columns(["conversations", "input", "generation_model", "generation_prompt", "raw_generation_responses", "generations", "row_index"])

In [32]:
ds_argilla_new

Dataset({
    features: ['conversations', 'input', 'generation_model', 'generation_prompt', 'raw_generation_responses', 'generations', 'row_index'],
    num_rows: 997984
})

In [14]:
ds_filtered

Dataset({
    features: ['topic', 'hash', 'category', 'language', 'model_name', 'title', 'source', 'custom_instruction', 'views', 'avatarUrl', 'skip_prompt_formatting', 'conversations', 'model', 'idx', 'system_prompt', 'id', 'row_index', 'input', 'conv-str'],
    num_rows: 998862
})

In [33]:
from tqdm import tqdm

for i in tqdm(range(len(ds_argilla_new)), total=len(ds_argilla_new)):
    if ds_argilla_new[i]["conversations"] != ds_filtered[ds_argilla_new[i]["row_index"]]["conversations"]:
        raise ValueError(f"Row {i} does not match")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 997984/997984 [02:14<00:00, 7440.18it/s]


In [34]:
ds_argilla_new

Dataset({
    features: ['conversations', 'input', 'generation_model', 'generation_prompt', 'raw_generation_responses', 'generations', 'row_index'],
    num_rows: 997984
})

In [35]:
ds_new_name = "argilla/OpenHermes-2.5-dpo"
#Old name: "argilla/OpenHermes-2.5-dpo-row-index-v2"
ds_argilla_new.push_to_hub(ds_new_name, split="train")


Uploading the dataset shards:   0%|          | 0/11 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/91 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/91 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/91 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/91 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/91 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/91 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/91 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/668 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/argilla/OpenHermes-2.5-dpo/commit/d5084bfd4effcba5aabf2bf3ee2b887f076f84d7', commit_message='Upload dataset', commit_description='', oid='d5084bfd4effcba5aabf2bf3ee2b887f076f84d7', pr_url=None, pr_revision=None, pr_num=None)